<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/ResNet_CIFAR10_functional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob, os, shutil
import numpy as np
import tensorflow as tf
from os.path import *
from tensorflow import keras
from tensorflow.keras import layers
from google.colab import drive, files

contentPth=os.getcwd()
drive.mount('/content/drive', force_remount=True)

drivePth = join(contentPth,  'drive')
myDrivePth = join(drivePth, 'My Drive')
tfModelsPth = join(myDrivePth, 'Tensorflow Models')

junkList=glob.glob('**')
for item in junkList:
    deletePth=os.path.join(contentPth, item)
    if exists(deletePth) and isdir(deletePth) and not deletePth.__contains__('drive'):
        shutil.rmtree(deletePth)
    elif exists(deletePth) and isfile(deletePth) and not deletePth.__contains__('h.5'):
        os.remove(deletePth)

print(contentPth)
print(drivePth)
print(myDrivePth)
print(f'{tfModelsPth}')

### A toy ResNet model

In addition to models with multiple inputs and outputs,
the functional API makes it easy to manipulate non-linear connectivity
topologies -- these are models with layers that are not connected sequentially,
which the `Sequential` API cannot handle.

A common use case for this is residual connections.
Let's build a toy ResNet model for CIFAR10 to demonstrate this:

In [ ]:
inputs = keras.Input(shape=(32, 32, 3), name="img")
x = layers.Conv2D(32, 3, activation="relu")(inputs)
x = layers.Conv2D(64, 3, activation="relu")(x)
block_1_output = layers.MaxPooling2D(3)(x)

x = layers.Conv2D(64, 3, activation="relu", padding="same")(block_1_output)
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
block_2_output = layers.add([x, block_1_output])

x = layers.Conv2D(64, 3, activation="relu", padding="same")(block_2_output)
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
block_3_output = layers.add([x, block_2_output])

x = layers.Conv2D(64, 3, activation="relu")(block_3_output)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10)(x)

model = keras.Model(inputs, outputs, name="ResNet_CIFAR10")
# model.summary()
model.name

Plot the model:

In [ ]:
keras.utils.plot_model(model, "Resnet_CIFAR10.png",
                       show_shapes=True,
                       show_dtype=True,
                       expand_nested=True,
                       dpi=256)

sourcePth=join(contentPth, 'ResNet_CIFAR10.png')
destinationPth=join(tfModelsPth, 'ResNet_CIFAR10.png')
if exists(sourcePth):
    shutil.copy2(sourcePth, desitnationPth)

Now train the model:

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["acc"],
)
# We restrict the data to the first [:1000] samples so as to limit execution time
# on Colab. Try to train on the entire dataset until convergence!
model.fit(x_train, y_train, batch_size=64, epochs=1, validation_split=0.2)  
model.save('ResNet_CIFAR10_model.h5')

loss,acc = model.evaluate(x_test, y_test, batch_size=64)

In [ ]:
sourcePth=join(contentPth, 'ResNet_CIFAR10_model.h5')
desitnationPth=join(tfModelsPth, 'ResNet_CIFAR10_model.h5')

shutil.copy2(sourcePth, desitnationPth)

model.load_weights(desitnationPth)
model.fit(x_train, y_train, batch_size=64, epochs=31, validation_split=0.2)
print()
loss, acc = model.evaluate(x_test, y_test)
print(f'\naccuracy: {round(acc * 100, 2)} percent') # 78.18 percent accuracy
model.save('ResNet_CIFAR10_model.h5')

In [ ]:
from matplotlib import pyplot as plt
img=x_test[0]
